### Configuration of the environment

In [ ]:
%tensorflow_version 2.x
!pip3 install --upgrade pip
#!pip install -qU t5
!pip3 install git+https://github.com/google-research/text-to-text-transfer-transformer.git #extra_id_x support

import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds

import t5

#Set the base dir(Google cloud bucket)
BASE_DIR = "gs://bucket_code_completion" 

if not BASE_DIR or BASE_DIR == "gs://":
  raise ValueError("You must enter a BASE_DIR.")
ON_CLOUD = True


if ON_CLOUD:
  import tensorflow_gcs_config
  from google.colab import auth
  # Set credentials for GCS reading/writing from Colab and TPU.
  TPU_TOPOLOGY = "2x2"
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    TPU_ADDRESS = tpu.get_master()
    print('Running on TPU:', TPU_ADDRESS)
  except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
  auth.authenticate_user()
  tf.config.experimental_connect_to_host(TPU_ADDRESS)
  tensorflow_gcs_config.configure_gcs_from_colab_auth()

tf.disable_v2_behavior()

# Improve logging.
from contextlib import contextmanager
import logging as py_logging

if ON_CLOUD:
  tf.get_logger().propagate = False
  py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

     |████████████████████████████████| 1.5MB 6.9MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
  Cloning https://github.com/google-research/text-to-text-transfer-transformer.git to /tmp/pip-req-build-ntwojsew
  Running command git clone -q https://github.com/google-research/text-to-text-transfer-transformer.git /tmp/pip-req-build-ntwojsew
     |████████████████████████████████| 361 kB 7.2 MB/s 
     |████████████████████████████████| 65 kB 2.5 MB/s 
     |████████████████████████████████| 1.2 MB 9.5 MB/s 
     |████████████████████████████████| 3.4 MB 20.9 MB/s 
     |████████████████████████████████| 3.8 MB 50.0 MB/s 
     |████████████████████████████████| 1.8 MB 63.1 MB/s 
     |████████████████████████████████| 2.9 MB 72.9 MB/s 
     |████████████████████████████████| 883 kB 48.6 MB/s 
  Created wheel for t5: filename=t5-0.8.1-py3-none-any.whl size=221270 sha256=6288da428449ea97ab0829f40d502b6fa772bd7a4cc922ea

Instructions for updating:
non-resource variables are not supported in the long term


### Loading of the tsv files
We loaded the 6 tsv files, please be sure to upload them on the bucket and to copy the correct path

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_java_construct = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_java_construct.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/eval_java_construct.tsv',
}

num_nq_examples_java_construct = dict(train=750000, validation=104037)

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_android_construct = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_android_construct.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/eval_android_construct.tsv',
}

num_nq_examples_android_construct = dict(train=750000, validation=98793)

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_java_block = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_java_block.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/eval_java_block.tsv',
}

num_nq_examples_java_block = dict(train=298470, validation=38840)

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_android_block = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_android_block.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/eval_android_block.tsv',
}

num_nq_examples_android_block = dict(train=204580, validation=26503)

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_java_token = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_java_token.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/eval_java_token.tsv',
}

num_nq_examples_java_token = dict(train=750000, validation=214682)

In [ ]:
#Validation(train and test on the same dataset)

nq_tsv_path_android_token = {
    "train":      'gs://bucket_code_completion/T5_extension/ft_datasets/train_android_token.tsv',
    "validation": 'gs://bucket_code_completion/T5_extension/ft_datasets/eval_android_token.tsv',
}

num_nq_examples_android_token = dict(train=750000, validation=198281)

### Preprocess of the dataset
In this step we preprocess the dataset.  
You have to change the path to vocab files (*vocab_model_path* and *vocab_path*)
We're going to preprocess all the tsv file so that T5 can use them for HP tuning.

In [ ]:
from t5.data import postprocessors as t5_postprocessors
from t5.seqio import Feature,SentencePieceVocabulary


# # Set the path of sentencepiece model and vocab files
# # Must be the same used for the pre-trained phase
vocab_model_path = 'gs://bucket_code_completion/T5_extension/code.model'
vocab_path = 'gs://bucket_code_completion/T5_extension/code.vocab'


TaskRegistry = t5.data.TaskRegistry
TfdsTask = t5.data.TfdsTask


def get_default_vocabulary():
  return SentencePieceVocabulary(vocab_model_path, 100)

DEFAULT_OUTPUT_FEATURES = {
    "inputs": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True, required=False),

    "targets": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True)
}

JAVA CONSTRUCT

In [ ]:
def nq_java_construct(split, shuffle_files=True):
  # We only have one file for each split.
  del shuffle_files

   # Load lines from the text file as examples.

  ds = tf.data.TextLineDataset(nq_tsv_path_java_construct[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw train examples...")
for ex in tfds.as_numpy(nq_java_construct("train").take(5)):
  print(ex)

A few raw train examples...
{'input': b'public void updateLockdownExceptions(ManagedObjectReference _this, String[] users) throws AuthMinimumAdminPermission, RemoteException, RuntimeFault, UserNotFound { Argument[] params = new Argument[2]; params[0] = new Argument("_this", "ManagedObjectReference", _this); params[1] = new Argument("users", "String[]", users); getWsc().invoke( <extra_id_0>); }', 'output': b'"UpdateLockdownExceptions", params, null'}
{'input': b'@Override public Collection<AgentProjectInfo> collectDependencies(String folder) { if ( <extra_id_0>){ devDependencies = findDevDependencies(folder); } File yarnLock = new File(folder + fileSeparator + YARN_LOCK); boolean yarnLockFound = yarnLock.isFile(); Collection<DependencyInfo> dependencies = new ArrayList<>(); if (yarnLockFound){ dependencies = parseYarnLock(yarnLock); } else { npmLsFailureStatus = true; } return getSingleProjectList(dependencies); }', 'output': b'!includeDevDependencies'}
{'input': b'@Override public Coll

In [ ]:
def java_construct_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['JAVA_CONSTRUCT:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('java_construct')
t5.data.TaskRegistry.add(
    "java_construct",
    dataset_fn=nq_java_construct,
    splits=["train", "validation"],
    text_preprocessor=[java_construct_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_java_construct
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("java_construct")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


A few preprocessed training examples...
{'inputs_pretokenized': b'JAVA_CONSTRUCT:public static boolean defaultTransactionSupported(final String persistenceUnit, final KunderaMetadata kunderaMetadata) { PersistenceUnitMetadata puMetadata = KunderaMetadataManager.getPersistenceUnitMetadata(kunderaMetadata, persistenceUnit); String txResource = puMetadata.getProperty(PersistenceProperties.KUNDERA_TRANSACTION_RESOURCE); if ( <extra_id_0>) { return true; } else if (txResource.isEmpty()) { throw new IllegalArgumentException("Property " + PersistenceProperties.KUNDERA_TRANSACTION_RESOURCE + " is blank"); } else { return false; } }', 'inputs': array([    3,  7641,    15,  5071, 17558,    56,  4569,    48,    45,
         289,   737,  2170,     5,    64,    26,     3, 25534,     9,
          44,  2289, 13550,   184,   411,     3, 30065,   411,     8,
           7,     3, 27457,   411, 16380,   411,    11,  2289, 13550,
         184,   411,   121,     4,    33, 27457,   411,     5, 30065,
      

JAVA TOKEN

In [ ]:
def nq_java_token(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_java_token[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw valid examples...")
for ex in tfds.as_numpy(nq_java_token("validation").take(5)):
  print(ex)

A few raw valid examples...
{'input': b'public void addThirdPartyPaymentWorkflow(com.mozu.api.contracts.sitesettings.order.ExternalPaymentWorkflowDefinition definition) throws Exception { MozuClient client = com.mozu.api.clients.commerce.settings.checkout.PaymentSettingsClient <extra_id_0> client.setContext(_apiContext); client.executeRequest(); client.cleanupHttpConnection(); }', 'output': b'.addThirdPartyPaymentWorkflowClient( definition);'}
{'input': b'public void addThirdPartyPaymentWorkflow(com.mozu.api.contracts.sitesettings.order.ExternalPaymentWorkflowDefinition definition) throws Exception { MozuClient client = com.mozu.api.clients.commerce.settings.checkout.PaymentSettingsClient.addThirdPartyPaymentWorkflowClient( definition); client.setContext(_apiContext <extra_id_0> client.executeRequest(); client.cleanupHttpConnection(); }', 'output': b');'}
{'input': b'public void addThirdPartyPaymentWorkflow(com.mozu.api.contracts.sitesettings.order.ExternalPaymentWorkflowDefinition def

In [ ]:
def java_token_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['JAVA_TOKEN:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('java_token')
t5.data.TaskRegistry.add(
    "java_token",
    dataset_fn=nq_java_token,
    splits=["train", "validation"],
    text_preprocessor=[java_token_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_java_token
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("java_token")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


A few preprocessed training examples...
{'inputs_pretokenized': b'JAVA_TOKEN:private List<ValidationError> validate(ExecutionInput executionInput, Document <extra_id_0> InstrumentationContext<List<ValidationError>> validationCtx = instrumentation.beginValidation(new InstrumentationValidationParameters(executionInput, document, graphQLSchema, instrumentationState)); Validator validator = new Validator(); List<ValidationError> validationErrors = validator.validateDocument(graphQLSchema, document); validationCtx.onCompleted(validationErrors, null); return validationErrors; }', 'inputs': array([    3,  7641,    15,  2591,    56,  8797,    85,    25,  8927,
          29,   858,     5,  1184,   488,  2717,   488,     9,  1168,
       32099, 13539,    92,    25,    71,    25,  8927,   243,  3756,
        4096,    11, 18230,     4,  3090,  1505,     5,    74, 13539,
        1505,   542,     5,  4713,   488,     9,   983,     9,  1985,
           2,   215,   645,     9, 18230,   119,    79,  90

JAVA BLOCK

In [ ]:
def nq_java_block(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_java_block[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw valid examples...")
for ex in tfds.as_numpy(nq_java_block("validation").take(5)):
  print(ex)

A few raw valid examples...
{'input': b'public Object createField(String name) { Object reader = _fieldMap.get(name); if (reader == null) <extra_id_0> return reader; }', 'output': b'reader = NullFieldDeserializer.DESER;'}
{'input': b'@Nonnull public static JSInvocation invoke (@Nonnull final JQueryInvocation aJQueryInvocation, @Nonnull final JSAssocArray aOptions) <extra_id_0>', 'output': b'{ return invoke (aJQueryInvocation).arg (aOptions); }'}
{'input': b'public static String presentMinMaxCount(long minmax) { if (minmax == Long.MAX_VALUE || minmax == Long.MIN_VALUE) <extra_id_0> return String.valueOf(minmax); }', 'output': b'{ return UNDEF_STRING; }'}
{'input': b'@Override public NonBottomTypeNode<ElkClass, ElkNamedIndividual> getCreateNode( final Collection<? extends ElkClass> members) <extra_id_0>', 'output': b'{ return getCreateUpdateableTypeNode( classTaxonomy_.getCreateNode(members)); }'}
{'input': b'public static JsonException typeMismatch(Object indexOrName, Object actual, Str

In [ ]:
def java_block_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['JAVA_BLOCK:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('java_block')
t5.data.TaskRegistry.add(
    "java_block",
    dataset_fn=nq_java_block,
    splits=["train", "validation"],
    text_preprocessor=[java_block_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_java_block
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("java_block")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


A few preprocessed training examples...
{'inputs_pretokenized': b'JAVA_BLOCK:public List<FileMetaData> getFiles(int level) { if (level == 0) { return level0.getFiles(); } else <extra_id_0> }', 'inputs': array([    3,  7641,    15,  3517,    56,  4569,    85,    25,   104,
        1598,    29, 19493,     5,    53,   694,     8,     7,    21,
          17,  1544,    40,   178,     7,    14,   694,   148,     4,
          33,   778,    18,     6,    77, 32099,     6,     1],
      dtype=int32), 'targets_pretokenized': b'{ return levels.get(level - 1).getFiles(); }', 'targets': array([    7,    14, 15313,     4,    33,     5,  1544,   139,   637,
           4,    33,   778,    18,     6,     1], dtype=int32)}
{'inputs_pretokenized': b'JAVA_BLOCK:@Override protected boolean isValueExpectedRecursive(XExpression expr) { final EObject container = expr.eContainer(); if (container instanceof SarlBreakExpression) <extra_id_0> if (container instanceof SarlContinueExpression) { return false; } if (

ANDROID CONSTRUCT

In [ ]:
def nq_android_construct(split, shuffle_files=True):
  # We only have one file for each split.
  del shuffle_files

   # Load lines from the text file as examples.

  ds = tf.data.TextLineDataset(nq_tsv_path_android_construct[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw train examples...")
for ex in tfds.as_numpy(nq_android_construct("train").take(5)):
  print(ex)

A few raw train examples...
{'input': b'private void writeToFile(final String content) { if ( <extra_id_0>) { return; } try { System.err.println(content); File f = new File(OUTPUT_FILE); f.delete(); FileWriter fstream = new FileWriter(OUTPUT_FILE); BufferedWriter out = new BufferedWriter(fstream); out.write(content); out.close(); } catch (Exception e) { e.printStackTrace(); } }', 'output': b'!WRITE_TO_FILE'}
{'input': b'private void writeToFile(final String content) { if (!WRITE_TO_FILE) { return; } try { System.err.println(content); File f = new File(OUTPUT_FILE); f.delete(); FileWriter fstream = new FileWriter(OUTPUT_FILE); BufferedWriter out = new BufferedWriter(fstream); out.write(content); out.close(); } catch ( <extra_id_0>) { e.printStackTrace(); } }', 'output': b'Exception e'}
{'input': b'private void writeToFile(final String content) { if (!WRITE_TO_FILE) { return; } try { System.err.println(content); File f = new File( <extra_id_0>); f.delete(); FileWriter fstream = new FileW

In [ ]:
def android_construct_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['ANDROID_CONSTRUCT:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('android_construct')
t5.data.TaskRegistry.add(
    "android_construct",
    dataset_fn=nq_android_construct,
    splits=["train", "validation"],
    text_preprocessor=[android_construct_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_android_construct
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("android_construct")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


A few preprocessed training examples...
{'inputs_pretokenized': b'ANDROID_CONSTRUCT:public AltosMag(AltosLink link) throws InterruptedException, TimeoutException { this(); link.printf("M\\ "); for (;;) { String line = link.get_reply_no_dialog(5000); if ( <extra_id_0>) { throw new TimeoutException(); } if (parse_string(line)) break; } }', 'inputs': array([    3, 16446,    15,  5071, 17558,    56,  4569,     3,  5134,
        1286, 29652,     5,  5134,  1286,   651,  1548,     8,    42,
        1441,    38,     9,     3,  6702,     7,    23,    18,  1548,
           4,  7103,    28,   491,     2,     3,    46,    50,    17,
          13,    13,     8,     7,    26,   626,    11,  1548,     4,
          33,    15,  7911,    15,  1137,    15,  2472,     5,     2,
        8132,    21,    17, 32099,     8,     7,    78,    24,     3,
        6702,    18,     6,    21,    17,   655,    15,   383,     5,
         687,     8,     8,   591,    13,     6,     6,     1],
      dtype=int32), 'targe

ANDROID TOKEN

In [ ]:
def nq_android_token(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_android_token[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw valid examples...")
for ex in tfds.as_numpy(nq_android_token("validation").take(5)):
  print(ex)

A few raw valid examples...
{'input': b'public void register(Context context <extra_id_0> IntentFilter intentFilter = buildFilter(); try { register(context, intentFilter); }catch (IllegalArgumentException e){ Log.w(TAG, "Error registering receiver. Receiver maybe already registered, unregistering to register again."); unregister(context); register(context, intentFilter); } }', 'output': b') {'}
{'input': b'public void register(Context context) { IntentFilter intentFilter <extra_id_0> try { register(context, intentFilter); }catch (IllegalArgumentException e){ Log.w(TAG, "Error registering receiver. Receiver maybe already registered, unregistering to register again."); unregister(context); register(context, intentFilter); } }', 'output': b'= buildFilter();'}
{'input': b'public void register(Context context) { IntentFilter intentFilter = buildFilter(); try <extra_id_0> register(context, intentFilter); }catch (IllegalArgumentException e){ Log.w(TAG, "Error registering receiver. Receiver ma

In [ ]:
def android_token_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['ANDROID_TOKEN:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('android_token')
t5.data.TaskRegistry.add(
    "android_token",
    dataset_fn=nq_android_token,
    splits=["train", "validation"],
    text_preprocessor=[android_token_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_android_token
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("android_token")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


A few preprocessed training examples...
{'inputs_pretokenized': b'ANDROID_TOKEN:public void switchToScreen(final <extra_id_0> Validate.notNull(newScreen); final MyScreen screen = this.instantiateScreen(newScreen); assert (!(screen instanceof ParameterScreen)); this.switchControl(screen); }', 'inputs': array([    3, 16446,    15,  2591,    56,  4569,    20,     3, 16074,
        1260,     5,    64, 32099,  3109,     4,  2623,     5,    74,
        1260,    10,    44,  2945,  1260,  2230,    11,    23,     4,
       16170,  1260,     5,    74,  1260,    10,   419,   124,     5,
        3339,   166,  4414,  1260,    79,    23,     4,  6916,   863,
           5,  3339,    10,     6,     1], dtype=int32), 'targets_pretokenized': b'Screens newScreen) {', 'targets': array([   3, 1260,   22,   24, 1260,    8,    7,    1], dtype=int32)}
{'inputs_pretokenized': b'ANDROID_TOKEN:public GoogleAnalytics provideGoogleAnalytics(Application application) { GoogleAnalytics ga = GoogleAnalytics.getInstanc

ANDROID BLOCK

In [ ]:
def nq_android_block(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_android_block[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw valid examples...")
for ex in tfds.as_numpy(nq_android_block("validation").take(5)):
  print(ex)

A few raw valid examples...
{'input': b'public void update() { if(Gdx.input.isKeyPressed(Keys.LEFT))<extra_id_0> if(Gdx.input.isKeyPressed(Keys.RIGHT)){ bucket.x += 200 * Gdx.graphics.getDeltaTime(); } if (bucket.x < 0) bucket.x = 0; if (bucket.x > 320 - 64) bucket.x = 320 - 64; }', 'output': b'{ bucket.x -= 200 * Gdx.graphics.getDeltaTime(); }'}
{'input': b'public void update() { if(Gdx.input.isKeyPressed(Keys.LEFT)){ bucket.x -= 200 * Gdx.graphics.getDeltaTime(); } if(Gdx.input.isKeyPressed(Keys.RIGHT))<extra_id_0> if (bucket.x < 0) bucket.x = 0; if (bucket.x > 320 - 64) bucket.x = 320 - 64; }', 'output': b'{ bucket.x += 200 * Gdx.graphics.getDeltaTime(); }'}
{'input': b'public void update() { if(Gdx.input.isKeyPressed(Keys.LEFT)){ bucket.x -= 200 * Gdx.graphics.getDeltaTime(); } if(Gdx.input.isKeyPressed(Keys.RIGHT)){ bucket.x += 200 * Gdx.graphics.getDeltaTime(); } if (bucket.x < 0) <extra_id_0> if (bucket.x > 320 - 64) bucket.x = 320 - 64; }', 'output': b'bucket.x = 0;'}
{'input':

In [ ]:
def android_block_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['ANDROID_BLOCK:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [ ]:
t5.data.TaskRegistry.remove('android_block')
t5.data.TaskRegistry.add(
    "android_block",
    dataset_fn=nq_android_block,
    splits=["train", "validation"],
    text_preprocessor=[android_block_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_android_block
)

In [ ]:
nq_task = t5.data.TaskRegistry.get("android_block")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": 256, "targets": 256})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


A few preprocessed training examples...
{'inputs_pretokenized': b'ANDROID_BLOCK:public T getById(long itemId) { if (itemId != 0) { for (TimelinePage<T> page : pages) { for (T item : page.items) <extra_id_0> } } return getEmptyItem(); }', 'inputs': array([    3, 16446,    15,  3517,    56,  4569,   299,     3, 11955,
           5,   288,  9249,     8,     7,    21,    17, 15571,    49,
         178,     7,    50,    17,  8430,   410,    25,    70,    29,
         715,    58,  7457,     8,     7,    50,    17,    70,   323,
          58,   715,     4,  2495,     8, 32099,     6,     6,    14,
          41,  1178,   169,    18,     6,     1], dtype=int32), 'targets_pretokenized': b'{ if (item.getId() == itemId) { return item; } }', 'targets': array([   7,   21,   17,  440,    4,  428,   16,   40, 9249,    8,    7,
         14,  323,   13,    6,    6,    1], dtype=int32)}
{'inputs_pretokenized': b'ANDROID_BLOCK:private void processRowSelect(View view) { MessageObject message = null; if (vi

### Hyper Parameter tuning
You can run the HP tuning using the following cells.  
Please set the correct path of the variable *MODEL_DIR* (the path to save the pretrained model in), *PATH_GIN_FILE* (the gin file configuration for this HP tuning) and *PRETRAINED_DIR* (the folder that contains the pretrained model).  
**Keep attention** to change the *pretrained_model_dir* in finetune step (if you are starting the HP tuning from scratch you have to set the value *PRETRAINED_DIR*, if you are restarting the HP tuning from a previous saved checkpoint you have to set the value *MODEL_DIR*)

In [ ]:
def _rate_num_input_examples(task):
  if "train" in task.splits:
    return float(task.num_input_examples("train"))
  elif "validation" in task.splits:
    return float(task.num_input_examples("validation"))
  else:
    raise ValueError("Task %s does not have a train or validation split." % (task.name))


t5.data.MixtureRegistry.remove("all_tasks")
t5.data.MixtureRegistry.add(
    "all_tasks",
    ["java_construct", "java_token", "java_block", "android_construct", "android_token", "android_block"],
    default_rate=_rate_num_input_examples
     #default_rate=1.0
)

In [ ]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

starter_learning_rate = 0.01
end_learning_rate = 1e-06
decay_steps = 10000

learning_rate_fn = PolynomialDecay(
    starter_learning_rate,
    decay_steps,
    end_learning_rate,
    power=0.5)


MODEL_SIZE = "small" 

# Set the folder where the checkpoints and all the others information will be writed
MODEL_DIR = 'gs://bucket_code_completion/T5_extension/HP_TUNING/polynomial/model'

# Specify the pre-trained dir which must contain the pre-trained models, the operative_config.gin file and the checkpoint file as well
PRETRAINED_DIR='gs://bucket_code_completion/T5_extension/pretrained_with_masking'


model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 256, 16),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

tf.io.gfile.makedirs(MODEL_DIR)

model = t5.models.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    learning_rate_schedule = learning_rate_fn,
    sequence_length={"inputs": 256, "targets": 256},
    save_checkpoints_steps=5000,
    keep_checkpoint_max=keep_checkpoint_max if ON_CLOUD else None,
    iterations_per_loop=100,
)

In [ ]:
PATH_GIN_FILE = 'gs://bucket_code_completion/T5_extension/HP_TUNING/polynomial/operative_config.gin'

import gin

with gin.unlock_config():
    gin.parse_config_file(PATH_GIN_FILE)
    #RUN FINE-TUNING
    FINETUNE_STEPS = 100000
    model.finetune(
        mixture_or_task_name="all_tasks",
        pretrained_model_dir=MODEL_DIR,
        finetune_steps=FINETUNE_STEPS
    )


INFO:root:system_path_file_exists:gs://bucket_comment_completion/Matteo/HP_TUNING/polynomial/operative_config.gin
ERROR:root:Path not found: gs://bucket_comment_completion/Matteo/HP_TUNING/polynomial/operative_config.gin
INFO:root:system_path_file_exists:gs://bucket_comment_completion/Matteo/HP_TUNING/polynomial/model/operative_config.gin
ERROR:root:Path not found: gs://bucket_comment_completion/Matteo/HP_TUNING/polynomial/model/operative_config.gin


INFO:tensorflow:Using config: {'_model_dir': 'gs://bucket_comment_completion/Matteo/HP_TUNING/polynomial/model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.74.11.226:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.74.11.226:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.74.11.226:8470', '_evaluation_master': 'grpc

/usr/local/lib/python3.6/dist-packages/t5/seqio/preprocessors.py:65: UserWarning: Creating resources inside a function passed to Dataset.map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  _tokenize, num_parallel_calls=tf.data.experimental.AUTOTUNE)


INFO:tensorflow:num_cores_per_replica: 1
INFO:tensorflow:computation_shape: [1, 1, 1, 1]
INFO:tensorflow:num_replicas: 8
INFO:tensorflow:device_assignment.topology.device_coordinates: [[[0 0 0 0]
  [0 0 0 1]
  [1 0 0 0]
  [1 0 0 1]
  [0 1 0 0]
  [0 1 0 1]
  [1 1 0 0]
  [1 1 0 1]]]
INFO:tensorflow:device_assignment.core_assignment: [[[0 0 0 0]]

 [[0 0 0 1]]

 [[1 0 0 0]]

 [[1 0 0 1]]

 [[0 1 0 0]]

 [[0 1 0 1]]

 [[1 1 0 0]]

 [[1 1 0 1]]]
INFO:tensorflow:auto_logical_to_physical_tpu logical_shape=[8] physical_shape=[2, 2, 2]
INFO:tensorflow:auto_logical_to_physical_tpu logical_to_physical = [(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 1, 0), (1, 1, 1), (1, 0, 0), (1, 0, 1)]
INFO:tensorflow:SimdMeshImpl init: Shape[batch=8] LayoutRules{('batch', 'batch'), ('experts', 'batch'), ('vocab', 'model'), ('heads', 'model'), ('ensemble', 'ensemble'), ('d_ff', 'model')}
INFO:tensorflow:Device Assignment: <tensorflow.python.tpu.device_assignment.DeviceAssignment object at 0x7f6014898f60>
INF

### Evaluate the performances

In [ ]:
# Use a larger batch size for evaluation, which requires less memory.
model.batch_size = 512
model.eval(
    mixture_or_task_name="all_tasks",
    # mixture_or_task_name="all_tasks",
    checkpoint_steps=-1 #evaluate only last checkpoint
)